In [1]:
# annoyingly this library needs an old (3.10 version of python to install)
# c:\Program Files\Python310\python.exe -m pip install requests
# c:\Program Files\Python310\python.exe -m pip install python-dotenv
# c:\Program Files\Python310\python.exe -m pip install web3
# c:\Program Files\Python310\python.exe -m pip install eip712-structs 

import requests
import os
import time
from dotenv import load_dotenv
from random import randint

from eip712_structs import EIP712Struct, Address, Uint, Boolean, make_domain
from web3 import Web3
from eth_account import Account



In [2]:
# variables in .env need to be = AEVOKEY, AEVOSECRET, WALLETADDRESS, SIGNINGKEY
load_dotenv()
# import an instrument ids dictionary 
from instrumentIDLists import aevoInstIDList

aevoKey = os.getenv("AEVOKEY")
aevoSecret = os.getenv("AEVOSECRET")
walletAddress = os.getenv("WALLETADDRESS")
signingKey = os.getenv("SIGNINGKEY")

In [3]:
# main functions
baseURL = "https://api.aevo.xyz/"
publicHeaders = {"accept": "application/json"}
privateHeaders = {
    "accept": "application/json",
    "AEVO-KEY": aevoKey,
    "AEVO-SECRET": aevoSecret
    }
decimals = 10**6

def getAevo(urlSuffix, headers = publicHeaders):
    fullUrl = baseURL + urlSuffix
    response = requests.get(fullUrl, headers = headers)
    return(response.json())

def deleteAevo(urlSuffix, payld = None):
    thisHeader = privateHeaders
    if payld:
        thisHeader["content-type"] = "application/json"
    fullUrl = baseURL + urlSuffix
    # need to check if this works with payload = None which would happen with deleteOrderID
    response = requests.delete(fullUrl, json = payld, headers = thisHeader)
    return(response.json())

def postAevo(urlSuffix, payld):
    thisHeader = privateHeaders
    thisHeader["content-type"] = "application/json"
    print(thisHeader)
    fullUrl = baseURL + urlSuffix
    response = requests.post(fullUrl, json = payld, headers = thisHeader)
    print(response)
    return(response.json())

def getWtAvePriceAevo(size, orderbook):
    sizeLeft = float(size)
    wtAvePx = 0
    for i in range(len(orderbook)):
        if sizeLeft <= 0:
            break
        sizeUsed = min(sizeLeft, float(orderbook[i][1]))
        wtAvePx += float(orderbook[i][0]) * sizeUsed
        sizeLeft -= sizeUsed
    return (wtAvePx / (size - sizeLeft)) # just in case the order is bigger than the market depth

def getBidOfferForSizeAevo(size, orderbook):
    if (size <= 0):
        size = 1
    print(orderbook)
    return (getWtAvePriceAevo(size, orderbook["bids"]), getWtAvePriceAevo(size, orderbook["asks"]))

# Don't rename this
class Order(EIP712Struct):
    maker = Address()
    isBuy = Boolean()
    limitPrice = Uint(256)
    amount = Uint(256)
    salt = Uint(256)
    instrument = Uint(256)
    timestamp = Uint(256)

def generateAndSignOrder(isBuy, instrument, limitprice, size):
    instrumentID = int(aevoInstIDList[instrument])
    limitPrice=int(limitprice * decimals)
    amount=int(size * decimals)
    salt=randint(0,10**6)
    timestamp=int(time.time())
    order_struct=Order(maker=walletAddress,
                      isBuy=isBuy,
                      limitPrice=limitPrice,
                      amount=amount,
                      salt=salt,
                      instrument=instrumentID,
                      timestamp=timestamp)
    domain = make_domain(name="Aevo Mainnet", version="1", chainId=1)
    signable_bytes = Web3.keccak(order_struct.signable_bytes(domain=domain))
    signature = Account._sign_hash(signable_bytes, signingKey).signature.hex()
    placingOrder = {"maker": walletAddress,
                      "is_buy": isBuy,
                      "limit_price": limitPrice,
                      "amount": amount,
                      "salt": salt,
                      "instrument": instrumentID,
                      "timestamp": timestamp,
                      "signature": signature}
    return (placingOrder)



In [8]:
# Public API calls
# getAevo("assets") # this gets a list of all the underlyings - no extra args
# getAevo("index?asset=" + ticker) # retrieve timestamp and price of ticker (e.g. "BNB")
# getAevo("markets") # this gets the details of all assets
# ticker = "ETH"
# getAevo("markets?asset=" + ticker + "&instrument_type=PERPETUAL") # this gets the details of one asset
# instrument = "ETH-PERP"
# getAevo("orderbook?instrument_name=" + instrument) # get orderbook (e.g. ETH-PERP)
# getBidOfferForSizeAevo(10, getAevo("orderbook?instrument_name=ETH-PERP")) # get the bid and offer in 10

# Private API calls
# acct = getAevo("account", headers = privateHeaders) # gets account details
# print(acct["available_balance"]) # get available balance of margin in USD
# getAevo("portfolio", headers = privateHeaders) # gets active portfolio details
# getAevo("orders", headers = privateHeaders) # gets all open orders

# deleteAevo("orders-all") # deletes all orders across all products
# payld = {"asset": "ETH", "instrument_type": "PERPETUAL"}
# deleteAevo("orders-all",payld) # deletes all orders across just the payld asset
# orderID = "0xdfaabxxxxxxx"
# deleteAevo("orders/"+ orderID) # deletes a specific order

# place a limit order
# payld = generateAndSignOrder(isBuy= False, instrument= "SOL-PERP", limitprice=195, size=0.2) # signs order instructions and names things correctly - yes it's strange - blame them not me
# postAevo("orders", payld=payld) # posts an order with instructions from payld





1924.721452
